Importing Libraries

In [224]:
import json
import xmltodict
import os

Converting Time given to seconds as easy to operate with

In [225]:
def convertTime(curTime):
    curTime = curTime[:8]
    parts = curTime.split(':')
    return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])

Reading the given .lwf file and making the list of all given leases by embedding it to the dictionaries

In [238]:
xml_file = open("data4.lwf")
data = xmltodict.parse(xml_file.read())["lease-workload"]
xml_file.close()

maxNodes = int(data["site"]["nodes"]["node-set"]["@numnodes"])
strval = ""
requests = []
for request in data["lease-requests"]["lease-request"]:
    temp = {}
    temp["id"] = int(request["lease"]["@id"])
    temp["arrival"] = convertTime(request["@arrival"])
    temp["nodes"] = int(request["lease"]["nodes"]["node-set"]["@numnodes"])
    temp["start"] = convertTime(request["lease"]["start"]["exact"]["@time"])
    temp["duration"] = convertTime(request["lease"]["duration"]["@time"])
    temp["deadline"] = convertTime(request["lease"]["deadline"]["@time"])
    temp["slack"] = round(
        (temp["deadline"] - temp["start"]) / temp["duration"], 2)
    if(temp["deadline"]>=temp["start"]):
        requests.append(temp)

Sorting the leases based on arrival time

In [239]:
requests = sorted(requests, key=lambda d: d['arrival'])

Checking if the space with given time start and end time present in the given node or not

In [240]:
def check(sch,start,end):
    sch.sort()
    for i in range(len(sch)-1):
        if(sch[i][1]<=start and end<=sch[i+1][0]):
            return True
    return False


Finding the slot for given lease for the required number of nodes and if available appedning the given lease

In [241]:
def findSlot(req, sch):
    startTime = req["start"]
    deadline = req["deadline"]
    duration = req["duration"]
    nodes = req["nodes"]
    id=req["id"]
    found=False

    for i in range(len(sch)):
        for j in range(len(sch[i])-1):
            if(sch[i][j][1]>=startTime or startTime<=sch[i][j+1][0]):    #checkling if startime is grater than or equal to start time or less than next lease endtime  
                st=max(startTime,sch[i][j][1])                              #start time is max of start time and end time of previous end time
                if(st+duration<=sch[i][j+1][0] and st+duration<=deadline):      

                    available_nodes=[]                                  #available node for the scheduling for given start and duration

                    for x in range(len(sch)):
                        if check(sch[x],st,st+duration):                #checking the nodes for the given start and duration
                            available_nodes.append(x)

                    if len(available_nodes)>=nodes: 
                        
                        tmp=[st,st+duration,id]
                        for x in range(nodes):
                            sch[available_nodes[x]].append(tmp)
                            sch[available_nodes[x]].sort()
                        found=True
                        break
        
        if found:
            return True
    return False             

            

If the node space could not found then trying the rescheduling by sorting on the slack values

In [242]:
def reschedule(req,sc):

    dup = []
    count=0
    st=[0,0,0] 
    en=[1e9,1e9,1e9]

    for i in range(maxNodes):
        tmp=[]
        tmp.append(st)
        tmp.append(en)
        dup.append(tmp)

    nsl=[]
    st=req["start"]
    for i in range(len(sc)):
        for j in range(1,len(sc[i])-1):

           if(st<=sc[i][j][0]):
            nsl.append(sc[i][j][2])
           else:
            dup[i].append(sc[i][j])

    s=set(nsl)
    nsl=list(s)
    nsld=[]
    nsld.append(req)
    for i in range(len(requests)):
        for j in range(len(nsl)):

            if(requests[i]["id"]==nsl[j]):
                nsld.append(requests[i])
    nsld = sorted(nsld, key=lambda d: d['slack'])
    for i in range(len(nsld)):

        if not findSlot(nsld[i],dup):
            return sc,1   
    return dup,0


Creating the node spaces and and calling the findslot function and if could not schedule trying to reschedule

In [243]:
sc = []
count=0
st=[0,0,0] 
en=[1e9,1e9,1e9]

for i in range(maxNodes):
    tmp=[st,en]
    sc.append(tmp)


for i in range(len(requests)):
     if not findSlot(requests[i],sc):
        sc,didnt=reschedule(requests[i],sc)
        count+=didnt  

print("Total Leases:", len(requests))
print("Total Leases Scheduled:", len(requests) - count)
print("Scheduled Percentage:", ((len(requests) - count)/ len(requests))*100)
print("Not possible to schedule", count)
    


Total Leases: 43
Total Leases Scheduled: 24
Scheduled Percentage: 55.81395348837209
Not possible to schedule 19
